In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import GradientBoostingClassifier,AdaBoostClassifier,RandomForestClassifier
from sklearn.model_selection import train_test_split
import category_encoders as ce
from sklearn.model_selection import GridSearchCV,RandomizedSearchCV
from sklearn.metrics import roc_auc_score

In [58]:
df1=pd.read_csv('clean_game.csv')
df1.head(5)

,Unnamed: 0,winner,increment_code,white_rating,black_rating,opening_name
0,3,1,20+0,1439,1454,Queen's Pawn Game
1,5,0,10+0,1250,1002,Sicilian Defense
2,8,-1,15+0,1439,1392,Italian Game
3,9,1,10+0,1381,1209,Scandinavian Defense
4,13,-1,8+0,1381,1607,French Defense


In [59]:
df1.columns

Index(['Unnamed: 0', 'winner', 'increment_code', 'white_rating',
       'black_rating', 'opening_name'],
      dtype='object')

In [60]:
df1.drop('Unnamed: 0',axis=1,inplace=True)

In [61]:
df1.head(5)

,winner,increment_code,white_rating,black_rating,opening_name
0,1,20+0,1439,1454,Queen's Pawn Game
1,0,10+0,1250,1002,Sicilian Defense
2,-1,15+0,1439,1392,Italian Game
3,1,10+0,1381,1209,Scandinavian Defense
4,-1,8+0,1381,1607,French Defense


In [62]:
df2_x=df1.drop(['winner'],axis=1)
df2_y=df1['winner']

In [63]:
df2_x

,increment_code,white_rating,black_rating,opening_name
0,20+0,1439,1454,Queen's Pawn Game
1,10+0,1250,1002,Sicilian Defense
2,15+0,1439,1392,Italian Game
3,10+0,1381,1209,Scandinavian Defense
4,8+0,1381,1607,French Defense
...,...,...,...,...
10034,10+0,1328,1252,French Defense
10035,10+0,1243,1142,English Opening
10036,10+10,1237,1231,French Defense
10037,10+0,1219,1286,Queen's Pawn Game


In [64]:
encode=['increment_code','opening_name']
c_enc=ce.CountEncoder()

In [65]:
count_enc=c_enc.fit_transform(df2_x[encode])
count_enc

,increment_code,opening_name
0,313,905
1,5425,1970
2,918,713
3,5425,493
4,391,997
...,...,...
10034,5425,997
10035,5425,511
10036,229,997
10037,5425,905


In [66]:
df2_x=df2_x.join(count_enc.add_suffix("_count"))
df2_x

,increment_code,white_rating,black_rating,opening_name,increment_code_count,opening_name_count
0,20+0,1439,1454,Queen's Pawn Game,313,905
1,10+0,1250,1002,Sicilian Defense,5425,1970
2,15+0,1439,1392,Italian Game,918,713
3,10+0,1381,1209,Scandinavian Defense,5425,493
4,8+0,1381,1607,French Defense,391,997
...,...,...,...,...,...,...
10034,10+0,1328,1252,French Defense,5425,997
10035,10+0,1243,1142,English Opening,5425,511
10036,10+10,1237,1231,French Defense,229,997
10037,10+0,1219,1286,Queen's Pawn Game,5425,905


In [67]:
df_t=df2_x.drop(['opening_name','increment_code'],axis=1)
df_t

,white_rating,black_rating,increment_code_count,opening_name_count
0,1439,1454,313,905
1,1250,1002,5425,1970
2,1439,1392,918,713
3,1381,1209,5425,493
4,1381,1607,391,997
...,...,...,...,...
10034,1328,1252,5425,997
10035,1243,1142,5425,511
10036,1237,1231,229,997
10037,1219,1286,5425,905


In [68]:
train2_x,test2_x,train2_y,test2_y=train_test_split(df_t,df2_y,test_size=0.3,random_state=42)
clf_2=GradientBoostingClassifier().fit(train2_x,train2_y)
clf_2.score(test2_x,test2_y)

0.6085657370517928

In [69]:
param={'n_estimators':[5,50,100],
      'max_depth':[1,3,5,7],
      'learning_rate':[0.01,0.1,1],
      'max_features':['sqrt','log2']}

r_search = RandomizedSearchCV(estimator=GradientBoostingClassifier(),param_distributions=param,cv=5,n_jobs =-1,verbose = 3)
r_search.fit(train2_x,train2_y)

Fitting 5 folds for each of 10 candidates, totalling 50 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  24 tasks      | elapsed:   26.7s
[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed:   38.0s finished


RandomizedSearchCV(cv=5, error_score='raise-deprecating',
                   estimator=GradientBoostingClassifier(criterion='friedman_mse',
                                                        init=None,
                                                        learning_rate=0.1,
                                                        loss='deviance',
                                                        max_depth=3,
                                                        max_features=None,
                                                        max_leaf_nodes=None,
                                                        min_impurity_decrease=0.0,
                                                        min_impurity_split=None,
                                                        min_samples_leaf=1,
                                                        min_samples_split=2,
                                                        min_weight_fraction_leaf=0.0,
                     

In [70]:
r_search.best_params_

{'n_estimators': 100,
 'max_features': 'log2',
 'max_depth': 1,
 'learning_rate': 0.1}

In [71]:
clf_2=GradientBoostingClassifier(n_estimators= 100,
 max_features= 'log2',
 max_depth= 1,
 learning_rate= 0.1).fit(train2_x,train2_y)
clf_2.score(test2_x,test2_y)

0.5982735723771581

In [72]:
rf = RandomForestClassifier()
rf.fit(train2_x,train2_y)
rf.score(test2_x,test2_y)

C:\Users\Anurag\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


0.5899734395750332

In [73]:
grid_param = {
    "n_estimators" : [90,100,115,130],
    'criterion': ['gini', 'entropy'],
    'max_depth' : range(2,20,1),
    'min_samples_leaf' : range(1,10,1),
    'min_samples_split': range(2,10,1),
    'max_features' : ['auto','log2']
}
r_search = RandomizedSearchCV(estimator=rf,param_distributions=grid_param,cv=5,n_jobs =-1,verbose = 3)
r_search.fit(train2_x,train2_y)

Fitting 5 folds for each of 10 candidates, totalling 50 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  24 tasks      | elapsed:   12.0s
[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed:   19.9s finished


RandomizedSearchCV(cv=5, error_score='raise-deprecating',
                   estimator=RandomForestClassifier(bootstrap=True,
                                                    class_weight=None,
                                                    criterion='gini',
                                                    max_depth=None,
                                                    max_features='auto',
                                                    max_leaf_nodes=None,
                                                    min_impurity_decrease=0.0,
                                                    min_impurity_split=None,
                                                    min_samples_leaf=1,
                                                    min_samples_split=2,
                                                    min_weight_fraction_leaf=0.0,
                                                    n_estimators=10,
                                                    n_jobs=None,
  

In [74]:
r_search.best_params_

{'n_estimators': 115,
 'min_samples_split': 7,
 'min_samples_leaf': 6,
 'max_features': 'auto',
 'max_depth': 13,
 'criterion': 'entropy'}

In [75]:
rf = RandomForestClassifier(criterion='entropy', max_depth=13, max_features='auto', min_samples_leaf=6, min_samples_split=7, n_estimators=115)
rf.fit(train2_x,train2_y)
rf.score(test2_x,test2_y)

0.6248339973439575

In [76]:
import pickle
filename1 = 'RF_model.pickle'
pickle.dump(rf, open(filename1, 'wb'))
filename2 = 'count_enc.pickle'
pickle.dump(c_enc, open(filename2, 'wb'))

In [77]:
rf.score(train2_x,train2_y)

0.7560836772449124

In [122]:
enc=['increment_code', 'opening_name']
e=c_enc.transform(df[enc])

In [87]:
df=pd.DataFrame(np.array([['10+0','French Defense']]),columns=['increment_code', 'opening_name'])

In [125]:
e1=e.to_numpy()

In [101]:
x1=np.array([1400,1500])
x1

array([1400, 1500])

In [126]:
e1=e1.flatten()

In [127]:
e1

array([5425,  997], dtype=int64)

In [114]:
c23=np.concatenate([np.array([1400,1500]),e1])

In [115]:
c23.shape

(4,)

In [116]:
c24=rf.predict_proba([c23])

In [117]:
c24

array([[0.76784587, 0.00495024, 0.22720389]])

In [128]:
c24[0][2]

0.22720388527897775

In [121]:
df1

,winner,increment_code,white_rating,black_rating,opening_name
0,1,20+0,1439,1454,Queen's Pawn Game
1,0,10+0,1250,1002,Sicilian Defense
2,-1,15+0,1439,1392,Italian Game
3,1,10+0,1381,1209,Scandinavian Defense
4,-1,8+0,1381,1607,French Defense
...,...,...,...,...,...
10034,1,10+0,1328,1252,French Defense
10035,1,10+0,1243,1142,English Opening
10036,-1,10+10,1237,1231,French Defense
10037,1,10+0,1219,1286,Queen's Pawn Game


In [ ]:
df1['opening_name'].value_